In [38]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random 
import time 
import torch
# %pip install transformers
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, random_split, DataLoader, SequentialSampler, RandomSampler
from transformers import get_linear_schedule_with_warmup
from joblib import dump, load


torch.cuda.is_available()
cuda = torch.device('cuda')

In [26]:
def preprocess(path):
    
    """
    
    preprocess: takes path to single tweets.csv and cleans it
    
    params:
    
    - path: path to .csv file
    
    returns: 
    - x: cleaned data 
    - y: labels 
    
    use: only use on original train/val csv from kaggle 
    
    """ 

    dat = pd.read_csv(path)
    add_row = dat.columns.to_numpy()
    dat.loc[len(dat.index)] = add_row
    dat.columns = ["Index", "Source", "Sentiment", "Tweet"]

    x = pd.DataFrame(dat["Tweet"])
    y = pd.DataFrame(dat["Sentiment"])

    y.drop(dat[dat.isnull().any(axis=1)].index.to_numpy(), inplace=True)
    y.reset_index(inplace=True)
    y.drop("index", axis=1, inplace=True)
    x.drop(dat[dat.isnull().any(axis=1)].index.to_numpy(), inplace=True)
    x.reset_index(inplace=True)
    x.drop("index", axis=1, inplace=True)

    y = y.to_numpy()
    x = x.to_numpy()

    y[y == "Positive"] = 0
    y[y == "Negative"] = 1
    y[y == "Neutral"] = 2
    y[y == "Irrelevant"] = 3

    x = x.flatten()
    y = y.astype(int).flatten()

    return (x, y)

def tokenize(x, y):
    
    """ 
    tokenize: takes in .numpy data and tokenizes using BertToknizer 
    
    params: 
    - x: ,npy tweets
    - y: .npy labels 
    
    returns: 
    - input_ids: id tokens 
    - attention_masks: masks 
    - labels: labels 
    
    """
    
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

    input_ids = []
    attention_masks = []
    for sentence in x:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        input_ids.append(encoded_dict["input_ids"])
        attention_masks.append(encoded_dict["attention_mask"])
    input_ids = torch.cat(input_ids, dim=0).cuda()
    attention_masks = torch.cat(attention_masks, dim=0).cuda()
    labels = torch.tensor(y).type(torch.LongTensor).cuda()

    return (input_ids, attention_masks, labels)

def train_val_test(x, y, first_split=0.40, second_split=0.50, seed=256):
    
    """
    train_val_test: sped up function for train/test/splitting of cleaned .npy data 
    
    params: 
    - x: cleaned tweet.npy 
    - y: cleaned label.npy 
    - first_split: test split proportion for first split (train/unprocessed) 
    - second_split: test split proportion for second split of unprocessed (val/test)
    - seed: random seed for splitting 
    
    returns: 
    - dict of train/val/tst sets 
    
    """ 
    np.random.seed(seed)
    x_tr, x_mid, y_tr, y_mid = train_test_split(
        x, y, test_size=first_split, random_state=seed
    )
    x_val, x_tst, y_val, y_tst = train_test_split(
        x_mid, y_mid, test_size=second_split, random_state=seed
    )

    return {
        "x_tr": x_tr,
        "y_tr": y_tr,
        "x_val": x_val,
        "y_val": y_val,
        "x_tst": x_tst,
        "y_tst": y_tst,
    }

def to_loader(tokenize_output, batch_size=32, type="tr"):
    
    """ 
    to_loader: 
    - compiles tokenized data into tensor datasets
    - turns tensor datasets into torch data loaders of specified batch_size
    
    params: 
    - tokenize_output: outputs resulting from @tokenize function 
    - batch_size: batch size for dataloaders 
    - type: must be in ['tr', 'val', 'tst'] 
        -- determines what type of random sampler to use 
        
    returns: 
    - dataloader: torch dataloader  
    """
    
    dat = TensorDataset(tokenize_output[0], tokenize_output[1], tokenize_output[2])
    if type == "tr":
        dataloader = DataLoader(dat, sampler=RandomSampler(dat), batch_size=batch_size)
        return dataloader
    elif type in ["val", "tst"]:
        dataloader = DataLoader(
            dat, sampler=SequentialSampler(dat), batch_size=batch_size
        )
        return dataloader

def data_pipe(X, Y, first_split=0.40, second_split=0.50, seed=256, batch_size=32):
    
    """ 
    data_pipe: splits, tokenizes, and returns data in dataloaders 
    
    params: 
    - X: tweet.npy 
    - Y: label.npy 
    - first_split: see @train_val_test 
    - second_split: see @train_val_test 
    - seed: see @train_val_test 
    - batch_size: see @to_loader
    
    returns: 
    - train_loader: loader for train dat 
    - val_loader: loader for val dat 
    - tst_loader: loader for tst dat
    """
    
    splits = train_val_test(X, Y, first_split, second_split, seed)
    train_token = tokenize(splits["x_tr"], splits["y_tr"])
    val_token = tokenize(splits["x_val"], splits["y_val"])
    tst_token = tokenize(splits["x_tst"], splits["y_tst"])

    train_loader = to_loader(train_token, batch_size, type = "tr")
    val_loader = to_loader(val_token, batch_size, type = "val")
    tst_loader = to_loader(tst_token, batch_size, type = "tst")

    return (train_loader, val_loader, tst_loader)

def flat_accuracy(preds, labels):
    """ 
    flat_accuracy: computes accuracy between preds and labels 
    
    params: 
    - preds: predictions
    - labels: labels 
    
    returns: 
    - accuracy
    """
    pred_flat = np.argmax(preds.numpy(), axis=1).flatten()
    labels_flat = labels.numpy().flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def f1_score_func(preds, labels):
    """ 
    f1_score_func: computes weighted f1 between preds and labels 
    
    params: 
    - preds: predictions
    - labels: labels 
    
    returns: 
    - weighted f1
    """
    preds_flat = np.argmax(preds.numpy(), axis=1).flatten()
    labels_flat = labels.numpy().flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

def finetune(path, model, train_loader, val_loader, epochs=5, tolerance=0.01, patience = 2):
    
    """ 
    finetune: finetunes pretrained BertForSequenceClassification model on tweet data 
    
    params: 
    - path: path to store best model 
    - model: BertForSequenceClassificationModel 
    - train_loader: torch loader for train dat 
    - val_loader: torch loader for val dat 
    - epochs: number of times to iterate through data 
    - tolerance: decides how close a lower validation loss can be to a previous best 
    - patience: decides how long loss can be lower than previous best before function exits 
    
    returns: 
    - training_stats: dictionary of accuracy, f1, val loss, and training loss over epochs 
    
    """

    model = model
    train_loader = train_loader
    val_loader = val_loader

    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )

    seed_val = 256
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)

    training_stats = {"acc": [], "f1": [], "val_loss": [], 'training_loss': []}
    val_loss_tracker = 2e5
    tolerance = tolerance

    for epoch_i in range(0, epochs):
        print("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(train_loader):
            b_input_ids = batch[0].cuda()
            b_input_mask = batch[1].cuda()
            b_labels = batch[2].cuda()
            model.zero_grad()
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss = outputs.loss.cuda()
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_train_loss / len(train_loader)
        training_stats['training_loss'].append(avg_train_loss)
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        # evaluation
        model.eval()
        total_eval_accuracy = 0
        total_eval_f1 = 0
        total_eval_loss = 0
        nb_eval_steps = 0
        for batch in val_loader:
            b_input_ids = batch[0].cuda()
            b_input_mask = batch[1].cuda()
            b_labels = batch[2].cuda()

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                loss = outputs.loss.cuda()
                logits = outputs.logits.cuda()

            total_eval_loss += loss.item()
            total_eval_f1 += f1_score_func(logits.cpu(), b_labels.cpu())
            total_eval_accuracy += flat_accuracy(logits.cpu(), b_labels.cpu())

        avg_val_accuracy = total_eval_accuracy / len(val_loader)
        avg_val_f1 = total_eval_f1 / len(val_loader)
        avg_val_loss = total_eval_loss / len(val_loader)
        training_stats["acc"].append(avg_val_accuracy)
        training_stats["f1"].append(avg_val_f1)
        training_stats["val_loss"].append(avg_val_loss)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
        print("  F1: {0:.2f}".format(avg_val_f1))
        print("  Loss: {0:.2f}".format(avg_val_loss))
        if avg_val_loss < val_loss_tracker - tolerance:
            val_loss_tracker = avg_val_loss
            torch.save(model.state_dict(), path + "best_model.pth")
        elif avg_val_loss > val_loss_tracker + tolerance:
            print("Exiting. Current loss is worst that previous.")
            break
            
    return training_stats


def get_embeddings(model, loader):
    
    """ 
    get_embeddings: uses BertForSequenceClassificationModel model to generate embeddings for sentences
    
    params: 
    - model: BertForSequenceClassificationModel 
    - loader: desired data to generate embeddings for 
    
    returns: 
    - embeddings: sentence embeddings 
    - labels: labels 
    
    """
    
    torch.cuda.empty_cache()
    
    embedding_list = []
    label_list = []

    model.eval()

    for step, batch in enumerate(loader):
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_labels = batch[2].cuda()
        with torch.no_grad():
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                output_hidden_states=True,
            )
            hidden = outputs.hidden_states
            embedding = hidden[-1][:, 0, :]
            embedding_list.append(embedding.cpu())
            label_list.append(b_labels.cpu())

    embeddings = torch.cat(embedding_list, dim=0)
    labels = torch.cat(label_list, dim=0)

    return (embeddings, labels)

def plot_tune_loss(stats_dict, y_lim = [0.1,1]): 
    
    """
    plot_tune_loss: plots the stats_dict returned from @finetune 
    
    params: 
    - stats_dict: return from @finetune 
    - y_lim: y_limits on graph 
    
    returns: 
    - fig: the plot 
    """
    avg_val_loss = stats_dict['val_loss']
    avg_acc = stats_dict['acc']
    avg_f1 = stats_dict['f1']
    avg_train_loss = stats_dict['training_loss']
    epoch_list = range(1, len(avg_val_loss)+1)
    
    %matplotlib inline
    fig = figure(figsize=(15, 15), dpi=800)

    plt.rcParams.update({"font.size": 22})
    plt.title("FineTuned Bert Sequence Classifier")
    plt.xlabel("Epochs")
    plt.ylabel("%")
    plt.ylim(y_lim[0], y_lim[1])
    lw = 5
    
    plt.plot(
    list(epoch_list),
    list(avg_f1),
    linewidth=lw,
    alpha=0.2,
    color= 'darkgreen',
    label= "F1 Score",
    marker="o",
    )

    plt.plot(
        list(epoch_list),
        list(avg_acc),
        linewidth=lw,
        alpha=0.2,
        color= 'darkblue',
        label= "Accuracy",
        marker="o",
        )

    plt.plot(
        list(epoch_list),
        list(avg_val_loss),
        linewidth=lw,
        alpha=0.2,
        color= 'darkred',
        label= "Validation Loss",
        marker="o",
        )
    
    plt.plot(
        list(epoch_list),
        list(avg_train_loss),
        linewidth=lw,
        alpha=0.2,
        color= 'purple',
        label= "Training Loss",
        marker="o",
        )
    
    plt.legend()
    plt.grid()
    
    return fig

def plot_knn(metric_distances, y_lim = [0.1, 1]):
    
    """
    plot_knn: plots the netric dictionary returned from @finetune 
    
    params: 
    - metric_distances: metric dictionary generated by knn gridsearch
    - y_lim: y_limits on graph 
    
    returns: 
    - fig: the plot 
    """
    
    %matplotlib inline

    fig = figure(figsize=(15, 15), dpi=800)

    plt.rcParams.update({"font.size": 22})
    plt.title("Hidden State Features. Across KNN Distance Metrics and NNeighbors")
    plt.xlabel("Neighbors")
    plt.ylabel("F1")
    plt.ylim(y_lim[0], y_lim[1])
    lw = 5

    colors = {'euclidean': 'darkred', 'cityblock': 'darkblue', 'cosine':'darkgreen', 'correlation':'purple'}
    for j in ["euclidean", "cityblock", "cosine", "correlation"]:
        plt.plot(
        list(range(1,len(metric_distances[j])+1)),
        list(metric_distances[j]),
        linewidth=lw,
        alpha=0.2,
        color= colors[j],
        label= j,
        marker="o",
        )


    plt.legend()
    plt.grid()
            
    return fig 

def plot_svc(score_dict, y_lim = [0.1, 0.9], c = 2, c_range = range(-5,5)): 
    %matplotlib inline

    fig = figure(figsize=(15, 15), dpi=800)

    plt.rcParams.update({"font.size": 22})
    plt.title("Hidden State Features. Across SVM Kernel and Regularization")
    plt.xlabel("C (Base " + str(c) + ")")
    plt.ylabel("F1")
    plt.ylim(y_lim[0], y_lim[1])
    lw = 5

    colors = {'poly': 'darkred', 'rbf': 'darkblue', 'sigmoid':'darkgreen', 'linear':'purple'}
    for j in ["poly", "rbf", "sigmoid", "linear"]:
        plt.plot(
        list(c_range),
        list(score_dict[j]),
        linewidth=lw,
        alpha=0.2,
        color= colors[j],
        label= j,
        marker="o",
        )


    plt.legend()
    plt.grid()
            
    return fig 

    
            


In [31]:
# load data 
dat = np.load('data/tweets.npy', allow_pickle = True)
labs = np.load('data/labels.npy', allow_pickle = True)
(train_load, val_load, tst_load) = data_pipe(dat, labs)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
# fine tune
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4,
    output_attentions=False,
    output_hidden_states=False,
).cuda()
training_stats = finetune("./test_model/", model, train_load, val_load, epochs=10)

#plot fine tune 
fine_plot = plot_tune_loss(training_stats)
fine_plot.savefig('finetune.jpg', bbox_inches='tight')

#predict on test set 
torch.cuda.empty_cache()

total_eval_f1 = 0
total_eval_acc = 0

for batch in tst_load:
    b_input_ids = batch[0].cuda()
    b_input_mask = batch[1].cuda()
    b_labels = batch[2].cuda()

    with torch.no_grad():
        outputs = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels,
        )
        logits = outputs.logits.cuda()

    total_eval_f1 += f1_score_func(logits.cpu(), b_labels.cpu())
    total_eval_acc += flat_accuracy(logits.cpu(), b_labels.cpu())

print(total_eval_f1/len(tst_load)) #89.9994
print(total_eval_acc/len(tst_load)) #89.9989

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

======== Epoch 1 / 10 ========
  Average training loss: 0.91
  Accuracy: 0.75
  F1: 0.74
  Loss: 0.67
======== Epoch 2 / 10 ========
  Average training loss: 0.47
  Accuracy: 0.84
  F1: 0.84
  Loss: 0.44
======== Epoch 3 / 10 ========
  Average training loss: 0.24
  Accuracy: 0.87
  F1: 0.87
  Loss: 0.40
======== Epoch 4 / 10 ========
  Average training loss: 0.14
  Accuracy: 0.89
  F1: 0.89
  Loss: 0.40
======== Epoch 5 / 10 ========
  Average training loss: 0.10
  Accuracy: 0.90
  F1: 0.90
  Loss: 0.45
Exiting. Current loss is worst that previous.
0.8999490213823009
0.8998978322672352


In [34]:
# load weights and get embeddings for further classification

# del model  

torch.cuda.empty_cache()

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4,
    output_attentions=False,
    output_hidden_states=False,
).cuda()
model.load_state_dict(torch.load("./test_model/best_model.pth"))

(tr_embeddings, tr_labels) = get_embeddings(
    model, train_load
)
(val_embeddings, val_labels) = get_embeddings(
     model, val_load
)
(tst_embeddings, tst_labels) = get_embeddings(
    model, tst_load
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [35]:
#move embeddings to numpy/cpu
tr_embeddings = tr_embeddings.numpy()
tr_labels = tr_labels.numpy()
val_embeddings = val_embeddings.numpy()
val_labels = val_labels.numpy()
tst_embeddings = tst_embeddings.numpy()
tst_labels = tst_labels.numpy()

In [28]:
#KNN#

#grid search
metric_dict_distance = {"euclidean": [], "cityblock": [], "cosine": [], "correlation": []}
for j in ["euclidean", "cityblock", "cosine", "correlation"]:
    for k in range(1, 9):
        knn = KNeighborsClassifier(n_neighbors=k, metric=j).fit(tr_embeddings, tr_labels)
        preds = knn.predict(val_embeddings)
        score = f1_score(val_labels, preds, average = "weighted")
        metric_dict_distance[j].append(score)
np.save('metric_dict.npy', metric_dict_distance)
#plot 
knn_plot = plot_knn(metric_dict_distance, y_lim = [0.88, .91]) #max cosine, n_neighbors = 1, 90.849%
knn_plot.savefig('knn_plot.jpg', bbox_inches='tight')

#test on tst embeddings 
best_knn = KNeighborsClassifier(n_neighbors = 1, metric = 'cosine').fit(tr_embeddings, tr_labels)
best_preds = best_knn.predict(tst_embeddings)
best_score = f1_score(tst_labels, best_preds, average = 'weighted') #91.264% F1

NameError: name 'metric_dict_distances' is not defined

In [57]:
#SVM#
C = [np.float_power(2,i) for i in range(-5,5)]
kernels = ['poly', 'rbf', 'sigmoid'] 
kernel_score_dict = {"poly": [], "rbf": [], "sigmoid": []}

for kern in kernels: 
    for reg in C: 
        clf = SVC(C = reg, kernel = kern, max_iter = 10000).fit(tr_embeddings, tr_labels)
        preds = clf.predict(val_embeddings)
        score = f1_score(val_labels, preds, average = 'weighted') 
        kernel_score_dict[kern].append(score)
        
np.save('kernel_dict.npy', kernel_score_dict)

svc_scores = []
for rg in C: 
    clf = LinearSVC(C = rg).fit(tr_embeddings, tr_labels)
    preds = clf.predict(val_embeddings)
    score = f1_score(val_labels, preds, average = "weighted")
    svc_scores.append(score)
np.save('svc_scores.npy', svc_scores)

#load and plot 
svc_scores = np.load('svc_scores.npy', allow_pickle = True)
kernel_dict = np.load('kernel_dict.npy', allow_pickle = True).item()
svm_plot = plot_svc(kernel_dict, y_lim = [0.85, 0.89]) 
svm_plot.savefig('svm_plot.jpg', bbox_inches='tight') #best kernel poly, 2e4, 88.68%

#get tst results on tst embeddings 

best_svm = SVC(C = 2e4, kernel = 'poly', max_iter = 10000).fit(tr_embeddings, tr_labels)
best_preds = best_svm.predict(tst_embeddings) 
best_score = f1_score(tst_labels, best_preds, average = 'weighted')
best_score #89.29%

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [42]:
#RANDOM FOREST 
n_estimators = list(range(100, 1000, 100))
rf_scores = []
for i in n_estimators: 
    rf = RandomForestClassifier(n_estimators = i, n_jobs = -1).fit(tr_embeddings, tr_labels)
    preds = rf.predict(val_embeddings) 
    score = f1_score(val_labels, preds, average = 'weighted')
    rf_scores.append(score)

# rf = RandomForestClassifier

In [44]:
np.save('rf_scores.npy', rf_scores)